# Used Car Price Prediction

## 1) Problem statement.

* This dataset comprises used cars sold on cardehko.com in India as well as important features of these cars.
* If user can predict the price of the car based on input features.
* Prediction results can be used to give new seller the price suggestion based on market condition.

## 2) Data Collection.
* The Dataset is collected from scrapping from cardheko webiste
* The data consists of 13 column and 15411 rows.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings

warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
df = pd.read_csv(r"./data/cardekho_imputated.csv", index_col=[0])

In [3]:
df.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


## Data Cleaning
### Handling Missing values

* Handling Missing values 
* Handling Duplicates
* Check data type
* Understand the dataset

In [4]:
## Check Null Values
##Check features with nan value
df.isnull().sum()

car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [5]:
## Remove Unnecessary Columns
df.drop('car_name', axis=1, inplace=True)
df.drop('brand', axis=1, inplace=True)

In [6]:
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [7]:
df['model'].unique()

<StringArray>
[        'Alto',        'Grand',          'i20',     'Ecosport',
      'Wagon R',          'i10',        'Venue',        'Swift',
        'Verna',       'Duster',
 ...
     'Panamera',      'Alturas',       'Altroz',           'NX',
     'Carnival',            'C',           'RX',        'Ghost',
 'Quattroporte',       'Gurkha']
Length: 120, dtype: str

In [8]:
# ================================
# Getting All Different Types Of Features
# ================================

num_features = df.select_dtypes(include="number").columns.tolist()
print("Num of Numerical Features:", len(num_features))

cat_features = df.select_dtypes(include=["object", "string", "category"]).columns.tolist()
print("Num of Categorical Features:", len(cat_features))

# Discrete = small number of unique values in numeric columns
discrete_features = [col for col in num_features if df[col].nunique() <= 25]
print("Num of Discrete Features:", len(discrete_features))

continuous_features = [col for col in num_features if col not in discrete_features]
print("Num of Continuous Features:", len(continuous_features))


Num of Numerical Features: 7
Num of Categorical Features: 4
Num of Discrete Features: 2
Num of Continuous Features: 5


In [9]:
from sklearn.model_selection import train_test_split

X = df.drop(['selling_price'], axis=1)
y = df['selling_price']


In [10]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


## Feature Encoding and Scaling
**One Hot Encoding for Columns which had lesser unique values and not ordinal**
* One hot encoding is a process by which categorical variables are converted into a form that could be provided to ML algorithms to do a better job in prediction.

In [11]:
len(df['model'].unique())

120

In [12]:
df['model'].value_counts()

model
i20             906
Swift Dzire     890
Swift           781
Alto            778
City            757
               ... 
Altroz            1
C                 1
Ghost             1
Quattroporte      1
Gurkha            1
Name: count, Length: 120, dtype: int64

In [13]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X["model"] = le.fit_transform(X["model"])


In [14]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,7,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,54,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,118,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,7,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,38,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [15]:
len(df['seller_type'].unique()),len(df['fuel_type'].unique()),len(df['transmission_type'].unique())

(3, 5, 2)

In [16]:
onehot_columns = ['seller_type', 'fuel_type', 'transmission_type']

num_features = [col for col in X.columns if col not in onehot_columns]


In [17]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(drop='first')

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", oh_transformer, onehot_columns),
        ("num", numeric_transformer, num_features)
    ],
    remainder='passthrough'
)


In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [19]:
X_train = preprocessor.fit_transform(X_train)
X_test  = preprocessor.transform(X_test)


In [20]:
print(X_train.shape)
print(X_test.shape)

(12328, 14)
(3083, 14)


In [21]:
pd.DataFrame(X_train)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.255968,0.323969,0.349100,-2.050819,1.756765,2.681685,-0.403824
1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.789199,-1.337798,-1.069394,0.985661,-0.547081,-0.382744,-0.403824
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.242618,-1.337798,-1.163564,-0.177042,0.893542,3.296910,-0.403824
3,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.022962,0.323969,0.178369,-0.465315,0.024564,0.396229,-0.403824
4,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.503081,1.321030,0.585469,0.149668,-0.550917,-0.502047,-0.403824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12323,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.556193,0.323969,1.702310,0.248161,-0.453086,-0.270460,2.070500
12324,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.899027,1.653383,0.084198,-0.876105,0.218310,0.066393,-0.403824
12325,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.036312,0.323969,-0.833967,0.185702,-0.932654,-0.779483,-0.403824
12326,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.530538,-1.337798,-0.951680,-0.273133,-0.550917,-0.432805,-0.403824


## Model Training And Model Selection

In [22]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

In [23]:
# ================================
# Evaluation function
# ================================
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse)
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [24]:
#================================
# Models
# ================================
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(max_iter=5000),
    "Ridge": Ridge(max_iter=5000),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest Regressor": RandomForestRegressor(random_state=42)
}

In [25]:
# ================================
# Training loop
# ================================
for name, model in models.items():

    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred  = model.predict(X_test)

    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(name)

    print("Model performance for Training set")
    print(f"- RMSE: {model_train_rmse:.4f}")
    print(f"- MAE:  {model_train_mae:.4f}")
    print(f"- R2:   {model_train_r2:.4f}")

    print("----------------------------------")

    print("Model performance for Test set")
    print(f"- RMSE: {model_test_rmse:.4f}")
    print(f"- MAE:  {model_test_mae:.4f}")
    print(f"- R2:   {model_test_r2:.4f}")

    print("=" * 35)
    print()

Linear Regression
Model performance for Training set
- RMSE: 553855.6665
- MAE:  268101.6071
- R2:   0.6218
----------------------------------
Model performance for Test set
- RMSE: 502543.5930
- MAE:  279618.5794
- R2:   0.6645

Lasso
Model performance for Training set
- RMSE: 553855.6710
- MAE:  268099.2286
- R2:   0.6218
----------------------------------
Model performance for Test set
- RMSE: 502542.7362
- MAE:  279614.7567
- R2:   0.6645

Ridge
Model performance for Training set
- RMSE: 553856.3158
- MAE:  268060.0140
- R2:   0.6218
----------------------------------
Model performance for Test set
- RMSE: 502534.4661
- MAE:  279557.4540
- R2:   0.6645

K-Neighbors Regressor
Model performance for Training set
- RMSE: 321504.2935
- MAE:  92469.1272
- R2:   0.8726
----------------------------------
Model performance for Test set
- RMSE: 257928.1083
- MAE:  113063.6150
- R2:   0.9116

Decision Tree
Model performance for Training set
- RMSE: 20797.2352
- MAE:  5164.8199
- R2:   0.9995


In [26]:
knn_params = {"n_neighbors": [2, 3, 10, 20, 40, 50]}

rf_params = {
    "max_depth": [5, 8, 15, None, 10],
    "max_features": [5, 7, "sqrt", 8],   # auto → sqrt
    "min_samples_split": [2, 8, 15, 20],
    "n_estimators": [100, 200, 500, 1000]
}


In [27]:
from sklearn.model_selection import RandomizedSearchCV

randomcv_models = [
    ('KNN', KNeighborsRegressor(), knn_params),
    ('RF', RandomForestRegressor(random_state=42), rf_params)
]

model_param = {}

for name, model, params in randomcv_models:
    random = RandomizedSearchCV(
        estimator=model,
        param_distributions=params,
        n_iter=100,
        cv=3,
        verbose=2,
        n_jobs=-1,
        random_state=42
    )

    random.fit(X_train, y_train)
    model_param[name] = random.best_params_

for model_name in model_param:
    print(f"---------------- Best Params for {model_name} -------------------")
    print(model_param[model_name])


Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] END ......................................n_neighbors=2; total time=   0.1s
[CV] END ......................................n_neighbors=2; total time=   0.0s
[CV] END ......................................n_neighbors=3; total time=   0.1s
[CV] END ......................................n_neighbors=2; total time=   0.1s
[CV] END ......................................n_neighbors=3; total time=   0.1s
[CV] END .....................................n_neighbors=10; total time=   0.1s
[CV] END ......................................n_neighbors=3; total time=   0.1s
[CV] END .....................................n_neighbors=10; total time=   0.1s
[CV] END .....................................n_neighbors=10; total time=   0.1s
[CV] END .....................................n_neighbors=20; total time=   0.1s
[CV] END .....................................n_neighbors=20; total time=   0.2s
[CV] END .....................................n_n

In [28]:
models = {
    "Random Forest Regressor": RandomForestRegressor(
        **model_param["RF"], random_state=42, n_jobs=-1
    ),

    "K-Neighbors Regressor": KNeighborsRegressor(
        **model_param["KNN"], n_jobs=-1
    )
}


In [29]:
for name, model in models.items():

    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred  = model.predict(X_test)

    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(name)

    print('Model performance for Training set')
    print(f"- RMSE: {model_train_rmse:.4f}")
    print(f"- MAE:  {model_train_mae:.4f}")
    print(f"- R2:   {model_train_r2:.4f}")

    print('----------------------------------')

    print('Model performance for Test set')
    print(f"- RMSE: {model_test_rmse:.4f}")
    print(f"- MAE:  {model_test_mae:.4f}")
    print(f"- R2:   {model_test_r2:.4f}")

    print('='*35)
    print()


Random Forest Regressor
Model performance for Training set
- RMSE: 136883.3460
- MAE:  54333.5003
- R2:   0.9769
----------------------------------
Model performance for Test set
- RMSE: 209651.7215
- MAE:  97259.1083
- R2:   0.9416

K-Neighbors Regressor
Model performance for Training set
- RMSE: 364471.5829
- MAE:  104532.0165
- R2:   0.8362
----------------------------------
Model performance for Test set
- RMSE: 265014.3947
- MAE:  118553.5923
- R2:   0.9067

